## Obrada knjige

In [1]:
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.lm.preprocessing import padded_everygram_pipeline
import ebooklib
import re
import nltk
from nltk.corpus import PlaintextCorpusReader
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from bs4 import BeautifulSoup
from ebooklib import epub
from nltk.lm import MLE
import os
import requests
import io

def epub2thtml(epub_path):
    book = epub.read_epub(epub_path)
    chapters = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            chapters.append(item.get_content())
    return chapters

blacklist = [   '[document]',   'noscript', 'header',   'html', 'meta', 'head','input', 'script']
def chap2text(chap):
    output = ''
    soup = BeautifulSoup(chap, 'html.parser')
    text = soup.find_all(text=True)
    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
    return output
def thtml2ttext(thtml):
    Output = []
    for html in thtml:
        text =  chap2text(html)
        Output.append(text)
    return Output
def epub2text(epub_path):
    chapters = epub2thtml(epub_path)
    ttext = thtml2ttext(chapters)
    return ttext

def listToString(s):
    str1 = ""
    for elem in s:
        str1 += elem
    return str1

out=epub2text('dostojevski_idiot.epub')

outString = listToString(out)
plainText = re.sub(r'[~%&\<>?#]+(r|n|xa0)',"",outString)
file1 = open("dostojevskiIdiot.txt","w")
file1.write(str(plainText))
file1.close()

## 1. i 2. dio zadatka

In [2]:

corpus_root = './'
newcorpus = PlaintextCorpusReader(corpus_root, "dostojevskiIdiot.txt")



mycorpus = [word.lower() for word in newcorpus.words()]
spl = 0.8*len(mycorpus)
#train = mycorpus[:int(spl)]
#test = mycorpus[int(spl):]

brojac=0
#moram tokenizirat
if os.path.isfile('dostojevskiIdiot.txt'):
    with io.open('dostojevskiIdiot.txt', encoding='utf8') as fin:
        brojac = brojac + 1
        if brojac <= spl:
            text = fin.read()
tokenized_text1 = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(text)]
tokenized_text=[]
for x in tokenized_text1:
    lista=[]
    for y in x:
        if(y.isalpha()):
            lista.append(y)
    tokenized_text.append(lista)
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)
model = MLE(n)
model.fit(train_data, padded_sents)

#model.vocab.lookup('oko devet sati ...'.split())
#model.vocab.lookup(tokenized_text[4]) # riječi iz treniranja
#print(model.counts)
#model.counts['dio']
#model.score('dio')

('u',
 'jednom',
 'od',
 'vagona',
 'trećega',
 'razreda',
 'našla',
 'se',
 'o',
 'svanuću',
 'sučelice',
 'jedan',
 'drugomu',
 'baš',
 'do',
 'prozora',
 'dva',
 'putnika',
 'obojica',
 'mladi',
 'ljudi',
 'obojica',
 'gotovo',
 'bez',
 'prtljage',
 'obojica',
 'negizdelinski',
 'odjevena',
 'obojica',
 's',
 'prilično',
 'zanimljivim',
 'fizionomijama',
 'a',
 'obojica',
 'su',
 'naposljetku',
 'poželjela',
 'zapodjenuti',
 'jedan',
 's',
 'drugim',
 'razgovor')

In [ ]:

num_test_grams = len(test_ngrams)
import random
r = random.sample(range(num_test_grams), 75)
test_gram_sample = [test_ngrams[i] for i in r]

In [7]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model,num_words,random_seed=42):
    content = []
    for token in model.generate(num_words,random_seed = random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

In [8]:
generate_sent(model,20,random_seed=7)

'grofica je krestovska također kažu otišla u inozemstvo'

In [9]:
generate_sent(model,28,random_seed=0)

'tako povjerljivo gledajući jevgenija pavloviča bili donekle su krivi general ivan fjodorovič jepančin stajao je kao zanesena ustala ali se nije provezla dalje od njih'

In [10]:
generate_sent(model,20,random_seed=1)

'reče on'

In [11]:
print(model.generate(28,text_seed=['oko','devet','sati']))

['i', 'dvadeset', 'minuta', 'govorio', 'je', 'jednom', 'u', 'životu', 'ukrao', 'štogod', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>']


## 3.dio zadatka

In [12]:
import dill as pickle

with open('ngram_modeldz2.pkl','wb') as fout:
    pickle.dump(model,fout)

In [13]:
with open('ngram_modeldz2.pkl','rb') as fin:
    model_loaded = pickle.load(fin)

In [14]:
generate_sent(model_loaded,20,random_seed = 42)

'opazio da je knez ruskim pismenima iz srednjega vijeka razvaline naše je društvo bilo zamišljeno sasvim jednostavno očekivali su jednu'